In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [2]:
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
X = np.load('drive/MyDrive/DATA/data_2/X.npy').astype(np.int8)
y = np.load('drive/MyDrive/DATA/data_2/y.npy').astype(np.int8)

In [5]:
scoring = {'accuracy': 'accuracy',
           'F1-score': make_scorer(f1_score, average='micro'),
           }

In [6]:
parameters = {'penalty': ['l1', 'l2'],
              'C':[0.01, 0.1, 1, 10, 100],
              'class_weight': [None, 'balanced']}
logreg = LogisticRegression(solver='liblinear')
gs = GridSearchCV(logreg, parameters, scoring, cv=5, verbose=4, refit='F1-score', n_jobs=-1)
gs.fit(X, y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'class_weight': [None, 'balanced'],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit='F1-score',
             return_train_score=False,
             scoring={'F1-score': make_scorer(f1_score, average=micro),
                

In [26]:
gs.best_score_

0.7532603285216526

In [20]:
y = np.where(y == 1, 0, 1)
np.unique(y)

array([0, 1])

In [21]:
scoring = {'accuracy': 'accuracy',
           'precision': 'precision',
           'recall': 'recall',
           'F1-score': 'f1',
           'recall -': make_scorer(recall_score, pos_label=0),
           'precision -': make_scorer(precision_score, pos_label=0, zero_division=0),
           }

scores_df = pd.DataFrame(index=['LogisticRegression'], columns=list(scoring.keys()))

In [22]:
clf = gs.best_estimator_
scores = cross_validate(clf, X, y, cv=5, scoring=scoring, n_jobs=-1)
mean_scores = pd.DataFrame(scores).apply(np.mean)

In [23]:
scores_df.loc[type(clf).__name__] = mean_scores.values[2:]
scores_df

,accuracy,precision,recall,F1-score,recall -,precision -
LogisticRegression,0.75326,0.752859,1,0.859006,0.00655738,0.4
